# Product Metrics with Pandas 🐼

You uploaded data from AppMetrica for the period from January 1 to March 31, 2020, only for users registered after January 1, 2020.

Data Description

- date – date of the event

- event - event

app_install - app installation<br>
app_start - open applications<br>
registration - register <br>
search – go to the product search page (catalogue)<br>
open_item - open item<br>
choose_item - send item to cart<br>
tap_basket - go to cart<br>
purchase - purchase confirmation<br>
- gender - gender of the user

- os_name - user platform

- city – user's city

- device_id - user device ID

- urm_source - the channel from which the user came

yandex-direct – Yandex direct <br>
google_ads - advertising on Google<br>
vk_ads - advertising in VK<br>
instagram_ads – ads on instagram<br>
facebook_ads - facebook ads<br>
referral - promotion "bring a friend"<br><br>
If there is ‘-’, then the channel is not defined or it is a direct download of the application or the visit is not from advertising
- purchase_sum - the cost of the purchase (when the 'purchase' event occurs)

Pay attention to the following:

- In the upload, only unique user actions for each day

- You can bypass the installation stage of the application if it was installed earlier

- It is possible to bypass the registration stage if the user was already logged in at the time of the session. However, unregistered users cannot make a purchase.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('C:/Users/ArtyomV/Downloads/KC_case_data.csv')

In [3]:
data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [4]:
data.isna().sum()

# Data looks solid!

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [5]:
data.shape

(2747968, 8)

In [6]:
data.date = pd.to_datetime(data.date)

# Date column to date format

# 1. MAU by Month
MAU in February:

In [7]:
data.groupby(data.date.dt.to_period('M'), as_index = True) \
    .agg({'device_id':'nunique'})

,device_id
date,
2020-01,99161
2020-02,75032
2020-03,74623


75032 MAU in February

# 2. Installs by Month
Number of installs in January:

In [8]:
data[(data.event == 'app_install')].date.dt.to_period('M').value_counts()

2020-01    80297
2020-02    38078
2020-03    36222
Freq: M, Name: date, dtype: int64

80297 install in January

# 3. Cohorts day by day: install -> purchase in 7 days
Assign cohorts to users by the day the app was installed and calculate the conversion from install to purchase for them within 7 days. Which cohort had the highest conversion rate? Enter the answer in the format: dd.mm.yyyy

Note: We consider a user to have converted if no more than 7 days have passed from the moment of installation to the first purchase.

In [9]:
import pandas as pd

# Load data into a dataframe
df = pd.read_csv('C:/Users/ArtyomV/Downloads/KC_case_data.csv')

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

data_install = df[data.event == 'app_install'][['date','device_id']]
data_install['install_date'] = data_install.date
data_install = data_install.drop(columns = 'date')
df = pd.merge(df,data_install, on = 'device_id', how = 'left')

# Create a cohort column based on the installation date
# df['cohort'] = df.groupby('device_id')['date'].transform('min').dt.strftime('%Y-%m-%d')


In [10]:
# Calculate the number of days between the installation date and purchase date for each user
df['days_since_install'] = (df['date'] - df.groupby('device_id')['date'].transform('min')).dt.days

# Filter only the rows with event = 'purchase'
purchases = df[df['event'] == 'purchase']

# Filter only the rows with days_since_install <= 7
converted = purchases[purchases['days_since_install'] <= 7]

# Group the data by cohort and calculate the conversion rate
conversion_rate = converted.groupby('install_date').agg({'device_id': 'nunique'}) / df.groupby('install_date').agg({'device_id': 'nunique'})


In [11]:
conversion_rate

,device_id
install_date,
2020-01-01,0.393406
2020-01-02,0.377226
2020-01-03,0.347211
2020-01-04,0.348990
2020-01-05,0.351287
...,...
2020-03-27,0.231026
2020-03-28,0.191567
2020-03-29,0.199642


In [12]:
conversion_rate.idxmax()[0]

Timestamp('2020-01-01 00:00:00')

2020-01-01 has the highest CR

# 4. CR for 2020-01-01
Enter the CR value from the previous question in % (round to 1 decimal place). The number format is with a dot. Example: 66.4.

In [13]:
round(conversion_rate.reset_index().query('install_date == "2020-01-01"').device_id*100,1)[0]

39.3

39.3% of CR in 2020-01-01

# 5. UTM New Users
Which paid marketing channel brought in the most new users?

In [14]:
data \
    .groupby('utm_source', as_index = True)\
    .agg({'device_id':'nunique'})\
    .query('utm_source != "-"') \
    .idxmax()[0]

'yandex-direct'

Yandex brought the most new users

# 6. Product Funnel CR
Analyze at what stage of the funnel most of the customers fall off. See the scenarios for registered and unregistered users separately. At what step the most registered users fall off?

In response, enter one of the steps, the conversion from which (to the next step) is the lowest, in the following format: "Search", "Adding a product", "Going to the cart", "Registration".

Hint:

- In order to highlight the group of those who need and do not need to register, add a column with the date of registration. If the <b>registration date is < the date of the event</b>, then the user is already registered.

In [15]:
data_install = data[data.event == 'app_install'][['date','device_id']]
data_install['install_date'] = data_install.date
data_install = data_install.drop(columns = 'date')
data = pd.merge(data,data_install, on = 'device_id', how = 'left')

# Get install_date (w/ join)

In [16]:
data['is_registered'] = data['install_date'] < data['date']

# Split by 2 registered groups

In [17]:
conversion_rate = data.groupby('is_registered', as_index = False)['event'].value_counts()

In [18]:
conversion_rate['prev_value'] = conversion_rate.groupby('is_registered')['count'].shift(1)

# Window function to get previous values

In [19]:
conversion_rate['cr_relative'] = conversion_rate['count'] / conversion_rate['prev_value']

# Relative CR

In [20]:
conversion_rate['cr_absolute'] = conversion_rate['count'] / conversion_rate.groupby('is_registered')['count'].transform('max')

# Absolute CT

In [21]:
conversion_rate = conversion_rate.drop(columns = 'prev_value')

In [22]:
conversion_rate

,is_registered,event,count,cr_relative,cr_absolute
0,False,app_start,294601,NaN,1.000000
1,False,search,279408,0.948429,0.948429
2,False,choose_item,214456,0.767537,0.727954
3,False,app_install,154597,0.720880,0.524767
4,False,tap_basket,153127,0.990491,0.519778
5,False,purchase,63480,0.414558,0.215478
6,False,register,57232,0.901575,0.194270
7,True,app_start,454104,NaN,1.000000
8,True,search,429231,0.945226,0.945226
9,True,choose_item,324213,0.755335,0.713962


tap_basket - Going from cart to purchase causes a low Conversion Rate

# 7. Purchase CR
Users who came from which channels showed the lowest conversion on their first purchase?

In response, enter one of the channels in the following format: "Yandex", "Google", "VK", "Facebook", "Instagram", "Referral program".

Hint:

- CR = number of users with first purchase /number of users which opened the App for the first time 

In [23]:
data.head(3)

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,is_registered
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,False
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,False
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,False


In [24]:
utm_purchases = data.query('event == "purchase"') \
                    .groupby('utm_source', as_index = True) \
                    .agg({'device_id':'nunique'}) \
                    .rename(columns = {'device_id':"CR_utm"})

In [25]:
utm_start = data.query('event == "app_start"') \
                .groupby('utm_source', as_index = True) \
                .agg({'device_id':'nunique'}) \
                .rename(columns = {'device_id':"CR_utm"})

In [26]:
utm_purchases / utm_start

,CR_utm
utm_source,
-,0.317556
facebook_ads,0.347382
google_ads,0.297674
instagram_ads,0.346658
referal,0.399523
vk_ads,0.362804
yandex-direct,0.295477


In [27]:
(utm_purchases / utm_start).idxmin()[0]

'yandex-direct'

Answer: Yandex showed the lowest conversion to purchase

# 8. UTM Median first check comparison
Users who came from which channel have a higher median first check? (only the first purchases of users are taken into account)

In response, enter one of the channels in the following format: "Yandex", "Google", "VK", "Facebook", "Instagram", "Referral program".

In [28]:
utm_median = data.sort_values(['device_id','date']).query('event == "purchase"')[['device_id','date','utm_source','purchase_sum']].copy()

In [29]:
utm_median['order_number'] = utm_median.groupby('device_id').date.rank()

In [30]:
utm_mediana = utm_median.query('order_number == 1 and utm_source !="-"') \
                        .groupby('utm_source', as_index = True) \
                        .agg(median_first_purchase = ('purchase_sum', 'median')) \
                        .sort_values('median_first_purchase', ascending = False)

utm_mediana

,median_first_purchase
utm_source,
referal,395.5
instagram_ads,393.5
vk_ads,393.0
yandex-direct,392.5
google_ads,390.5
facebook_ads,389.0


In [31]:
utm_mediana.idxmax()[0]

'referal'

Referal program has the biggest median first check

# 9. ROMI

Advertising spent data:

- Yandex - 10,491,707 rubles. <br>
- Google - 10,534,878 rubles. <br>
- Facebook - 8 590 498 rubles. <br>
- Instagram - 8 561626 rubles. <br>
- VK - 9 553 531 rubles.<br>
- Referral program costs: if a user brings a friend and the latter makes the first purchase, both will receive 100 rubles.

Which paid acquisition channel (among ads) has the highest ROMI?

In response, enter one of the channels in the following format: "Yandex", "Google", "VK", "Facebook", "Instagram".


ROMI = ("Gross Profit" - "Martketing Investment) / "Martketing Investment"

In [32]:
reward = 100
users = 2
referal_spent = data.query('utm_source == "referal"') \
                    .agg({'device_id':'nunique'})[0] * users * reward

# Calculate money spent on Referral program

In [33]:
d = {'vk_ads':9553531, 
              'yandex-direct':10491070, 
              'instagram_ads':8561626, 
              'google_ads':10534878, 
              'facebook_ads':8590498, 
              'referal': referal_spent}

marketing_investment = pd.DataFrame(list(d.items()), columns=['utm_source', 'purchase_sum'])
marketing_investment = marketing_investment.set_index('utm_source')

In [34]:
marketing_investment

,purchase_sum
utm_source,
vk_ads,9553531
yandex-direct,10491070
instagram_ads,8561626
google_ads,10534878
facebook_ads,8590498
referal,3185200


In [35]:
gross_profit = data.query('utm_source != "-" and utm_source != "referal"') \
                   .groupby('utm_source', as_index = True) \
                   .agg({'purchase_sum':'sum'})

In [36]:
ROMI = (gross_profit - marketing_investment) / marketing_investment

In [37]:
ROMI

,purchase_sum
utm_source,
facebook_ads,0.425983
google_ads,0.221493
instagram_ads,0.699090
referal,NaN
vk_ads,0.715560
yandex-direct,0.326401


In [38]:
ROMI.idxmax()[0]

'vk_ads'

VK Ads has the highest ROMI between paid channels